# Notebook Jupyter adapté pour le nettoyage et la transformation des jeu de données 

In [9]:
import json
import csv
import pandas as pd

### - Jeu de données ghostbikes (accident mortel)

##### 1. Transformation

In [8]:
# Fonction pour charger un fichier .geojson
def load_geojson(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# Fonction pour nettoyer les données
def clean_data(data):
    cleaned_data = []
    for feature in data.get("features", []):
        properties = feature.get("properties", {})
        geometry = feature.get("geometry", {})
        coordinates = geometry.get("coordinates", [])
        if len(coordinates) == 2:
            cleaned_data.append({
                "id": properties.get("@id"),
                "lat": coordinates[1],
                "lon": coordinates[0],
                "start_date": properties.get("start_date")
            })
    return cleaned_data

# Fonction pour écrire les données nettoyées dans un fichier CSV
def write_to_csv(cleaned_data, output_file):
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["id", "lat", "lon", "start_date"])
        writer.writeheader()
        writer.writerows(cleaned_data)

# Chemin d'entrée et de sortie
input_file = "export.geojson"
output_file = "ghostbikes.csv"

# Chargement, nettoyage et exportation
data = load_geojson(input_file)
cleaned_data = clean_data(data)
write_to_csv(cleaned_data, output_file)

##### 2. Finalisation

In [11]:
data_ghostbike = pd.read_csv("ghostbikes.csv", sep=",")
df_ghostbike = pd.DataFrame(data_ghostbike)
# Suppression de "node/" dans la colonne 'id'
df_ghostbike["id"] = df_ghostbike["id"].str.replace("node/", "")
# Ajout de la colonne 'statut_accident' avec la valeur "Mortel"
df_ghostbike["statut_accident"] = "Mortel"
print(df_ghostbike)

            id        lat       lon  start_date statut_accident
0   9611135117  45.180153  5.716451  2022-03-26          Mortel
1  11002892805  45.177253  5.718046  2023-06-24          Mortel


### - Jeu de données ghostbikes (accident mortel)